<a href="https://colab.research.google.com/github/ChapelFob80930/AI-Generated-Text-Detector/blob/main/AITextTest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing the Dataset**

In [ ]:
import numpy as np
import pandas as pd

ai_generated_dataset = pd.read_csv("/content/train_v2_drcat_02.csv")
print(ai_generated_dataset.shape)
ai_generated_dataset = ai_generated_dataset.dropna()
ai_generated_dataset = ai_generated_dataset.drop(columns=['prompt_name','source','RDizzl3_seven'])
print(ai_generated_dataset.head())
print(ai_generated_dataset.shape)

(44868, 5)
                                                text  label
0  Phones\n\nModern humans today are always on th...      0
1  This essay will explain if drivers should or s...      0
2  Driving while the use of cellular devices\n\nT...      0
3  Phones & Driving\n\nDrivers should not be able...      0
4  Cell Phone Operation While Driving\n\nThe abil...      0
(44868, 2)


In [ ]:
import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_eng to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package averaged_perceptron_tagger_rus to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_rus.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_d


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
# import spacy
# nlp = spacy.load('en_core_web_sm')

# def lemmatize_text(text):
#   doc = nlp(" ".join(text))
#   return [token.lemma_ for token in doc]

In [ ]:
# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')

# stop_words = set(stopwords.words('english'))

# def _remove_stopwords_(texts):
#   return [text for text in texts if text.isalnum() and text not in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# from nltk.tokenize import word_tokenize
# nltk.download('punkt')

# def preprocess_text(series):
#     def process(text):
#         tokens = word_tokenize(text)
#         tokens = _remove_stopwords_(tokens)
#         tokens = lemmatize_text(tokens)
#         return " ".join(tokens)
#     return series.apply(process)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#**Preprocessing Data**

In [ ]:
import numpy as np
import pandas as pd
import spacy
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import nltk


nlp = spacy.load('en_core_web_sm')


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


def preprocess(text):
    """Tokenize, remove stopwords, and lemmatize text using spaCy."""
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.text.lower() not in stop_words]
    return " ".join(tokens)


def parallel_preprocessing(series):
    """Preprocess text using spaCy, parallel processing, and tqdm for progress tracking."""
    tqdm.pandas()
    return Parallel(n_jobs=10, backend="multiprocessing")(delayed(preprocess)(text) for text in tqdm(series, desc="Preprocessing"))


ai_generated_dataset["preprocessedText"] = parallel_preprocessing(ai_generated_dataset["text"])

print(ai_generated_dataset.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Preprocessing:   0%|          | 0/44868 [00:00<?, ?it/s]

                                                text  label  \
0  Phones\n\nModern humans today are always on th...      0   
1  This essay will explain if drivers should or s...      0   
2  Driving while the use of cellular devices\n\nT...      0   
3  Phones & Driving\n\nDrivers should not be able...      0   
4  Cell Phone Operation While Driving\n\nThe abil...      0   

                                    preprocessedText  
0  phone Modern human today always phone always p...  
1  essay explain driver able use electronic devic...  
2  drive use cellular device today society though...  
3  Phones Driving driver able use phone operate v...  
4  Cell Phone Operation drive ability stay connec...  


In [ ]:
# train_essays["preprocessedText"]=preprocess_text(train_essays['text'])

In [ ]:
# ai_human_dataset["preprocessedText"]=preprocess_text(ai_human_dataset['text'])

In [ ]:
# print(train_essays.head())

In [ ]:
# print(ai_human_dataset.head())

In [ ]:
# from gensim.models import Word2Vec

# sentences = train_essays['preprocessedText'].apply(lambda x: x.split())

# word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# word_vector = word2vec_model.wv['car']
# print("Vector for 'car':", word_vector)

# # train_essays['word2vecVector'] = sentences.apply(
# #     lambda x: sum([word2vec_model.wv[word] for word in x if word in word2vec_model.wv]) / len(x)
# # )

# def calculate_average_word_vector(words,model,vector_size=100):
#     word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
#     if word_vectors:
#         return np.mean(word_vectors, axis=0)
#     else:
#         return np.zeros(vector_size)

# train_essays['word2vecVector'] = train_essays['preprocessedText'].apply(lambda x: calculate_average_word_vector(x.split(),word2vec_model))


In [ ]:
# from gensim.models import Word2Vec

# sentences = ai_human_dataset["preprocessedText"].apply(lambda x: x.split())

# word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# word_vector = word2vec_model.wv['car']
# print("Vector for 'car':", word_vector)

# # train_essays['word2vecVector'] = sentences.apply(
# #     lambda x: sum([word2vec_model.wv[word] for word in x if word in word2vec_model.wv]) / len(x)
# # )

# def calculate_average_word_vector(words,model,vector_size=100):
#     word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
#     if word_vectors:
#         return np.mean(word_vectors, axis=0)
#     else:
#         return np.zeros(vector_size)

# ai_human_dataset['word2vecVector'] = ai_human_dataset["preprocessedText"].apply(lambda x: calculate_average_word_vector(x.split(),word2vec_model))


#**Creating Word2Vec Embeddings**

In [ ]:
from gensim.models import Word2Vec

sentences = ai_generated_dataset["preprocessedText"].apply(lambda x: x.split())

word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

word_vector = word2vec_model.wv['car']
print("Vector for 'car':", word_vector)

# train_essays['word2vecVector'] = sentences.apply(
#     lambda x: sum([word2vec_model.wv[word] for word in x if word in word2vec_model.wv]) / len(x)
# )

def calculate_average_word_vector(words,model,vector_size=100):
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)

def parallel_word2vec(row):
    return calculate_average_word_vector(row.split(), word2vec_model)

# ai_generated_dataset['word2vecVector'] = ai_generated_dataset["preprocessedText"].apply(lambda x: calculate_average_word_vector(x.split(),word2vec_model))
ai_generated_dataset['word2vecVector'] = Parallel(n_jobs=10,backend='multiprocessing')(delayed(parallel_word2vec)(row) for row in tqdm(ai_generated_dataset["preprocessedText"], desc="Word2Vec"))
print(ai_generated_dataset.head())



Vector for 'car': [ 0.5962737  -0.92818195  2.6535997  -0.05764337  1.2243859   4.3238535
 -1.4641442   0.941546    3.5248525  -0.93311226  0.37283757 -1.2600119
 -0.91792876 -0.59376144  0.1407949  -0.55901426  2.8251555  -0.29198852
 -0.08630872  0.37593096  3.8258963   0.23630196  2.667015   -0.52166784
  0.76454     0.8136561  -1.9429241  -2.2136018  -0.5002454   1.8470684
 -0.71962917  0.06718119 -1.5099088  -0.41480696  0.44296107  0.06202271
 -0.55416137  0.7243642   0.75591224 -3.474041    0.6676011   0.6385082
  0.29403654  0.70356816  0.7629455  -1.8164337   0.35143366  0.29509485
  1.0757256   0.5113459   1.3205835  -0.48045835  0.8101994  -1.2591813
  0.29692215  3.040532    0.6535584   0.87207186 -1.6348126  -1.9509165
 -3.7222629  -1.1953813   3.9625354  -1.2378793  -0.49908292 -0.9195943
 -2.462364   -0.05109651 -0.7010699  -1.1108385   1.5739691  -2.1976953
 -0.17178956 -0.6267602  -1.5306085  -2.0007076  -3.48499    -0.34771413
  0.19476208 -1.0221956   2.2191246   0.6

Word2Vec:   0%|          | 0/44868 [00:00<?, ?it/s]

                                                text  label  \
0  Phones\n\nModern humans today are always on th...      0   
1  This essay will explain if drivers should or s...      0   
2  Driving while the use of cellular devices\n\nT...      0   
3  Phones & Driving\n\nDrivers should not be able...      0   
4  Cell Phone Operation While Driving\n\nThe abil...      0   

                                    preprocessedText  \
0  phone Modern human today always phone always p...   
1  essay explain driver able use electronic devic...   
2  drive use cellular device today society though...   
3  Phones Driving driver able use phone operate v...   
4  Cell Phone Operation drive ability stay connec...   

                                      word2vecVector  
0  [-0.6782628, 0.23650575, 0.46880054, 0.1871484...  
1  [-0.019365383, -0.0036262611, 0.18905105, -0.0...  
2  [0.004628463, 0.3914926, 0.5535957, 0.12398179...  
3  [0.23808499, -0.07736159, 0.7977521, -0.061762...  
4  [-0.17

In [ ]:
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(train_essays['preprocessedText'])]

# doc2vec_model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4)

# train_essays['doc2vecVector'] = train_essays.index.map(lambda x: doc2vec_model.dv[str(x)])


In [ ]:
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(ai_human_dataset["preprocessedText"])]

# doc2vec_model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4)

# ai_human_dataset['doc2vecVector'] = ai_human_dataset.index.map(lambda x: doc2vec_model.dv[str(x)])

#**Creating Doc2Vec Embeddings**

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


def prepare_tagged_documents(series):
    """Prepare tagged documents for Doc2Vec."""
    return [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(series)]


tqdm.pandas(desc="Preparing Tagged Documents")
tagged_data = prepare_tagged_documents(tqdm(ai_generated_dataset["preprocessedText"], desc="Tagging Data"))


doc2vec_model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4)

# Function to generate Doc2Vec vectors
def generate_doc2vec_vector(index):
    """Generate a Doc2Vec vector for a given index."""
    return doc2vec_model.dv[str(index)]

# Use tqdm and joblib to parallelize vector generation
def generate_doc2vec_vectors_parallel(dataframe):
    """Generate Doc2Vec vectors in parallel with progress tracking."""
    indices = dataframe.index
    return Parallel(n_jobs=10, backend="multiprocessing")(delayed(generate_doc2vec_vector)(index) for index in tqdm(indices, desc="Generating Doc2Vec Vectors"))

ai_generated_dataset['doc2vecVector'] = generate_doc2vec_vectors_parallel(ai_generated_dataset)


print(ai_generated_dataset.head())


Tagging Data:   0%|          | 0/44868 [00:00<?, ?it/s]

Generating Doc2Vec Vectors:   0%|          | 0/44868 [00:00<?, ?it/s]

                                                text  label  \
0  Phones\n\nModern humans today are always on th...      0   
1  This essay will explain if drivers should or s...      0   
2  Driving while the use of cellular devices\n\nT...      0   
3  Phones & Driving\n\nDrivers should not be able...      0   
4  Cell Phone Operation While Driving\n\nThe abil...      0   

                                    preprocessedText  \
0  phone Modern human today always phone always p...   
1  essay explain driver able use electronic devic...   
2  drive use cellular device today society though...   
3  Phones Driving driver able use phone operate v...   
4  Cell Phone Operation drive ability stay connec...   

                                      word2vecVector  \
0  [-0.6782628, 0.23650575, 0.46880054, 0.1871484...   
1  [-0.019365383, -0.0036262611, 0.18905105, -0.0...   
2  [0.004628463, 0.3914926, 0.5535957, 0.12398179...   
3  [0.23808499, -0.07736159, 0.7977521, -0.061762...   
4  [

In [ ]:
# print(train_essays.head())

In [ ]:
# from sklearn.model_selection import train_test_split

# y = train_essays['generated']

# train_index, test_index = train_test_split(train_essays.index, test_size=0.2, random_state=42)

#**Splitting the Test and Train indexes and Separating the Labels from the Data**

In [ ]:
from sklearn.model_selection import train_test_split

y = ai_generated_dataset['label']

train_index, test_index = train_test_split(ai_generated_dataset.index, test_size=0.2, random_state=42)

In [ ]:
# from sklearn.model_selection import train_test_split

# y = ai_human_dataset['generated']

# train_index, test_index = train_test_split(ai_human_dataset.index, test_size=0.2, random_state=42)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf_vectorizer = TfidfVectorizer(max_features=1000)
# X_tfidf = tfidf_vectorizer.fit_transform(train_essays['preprocessedText'])

# X_train_tfidf, X_test_tfidf = X_tfidf[train_index], X_tfidf[test_index]
# y_train, y_test = y[train_index], y[test_index]


#**Creating TF-IDF Embeddings**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(ai_generated_dataset["preprocessedText"])

X_train_tfidf, X_test_tfidf = X_tfidf[train_index], X_tfidf[test_index]
y_train, y_test = y[train_index], y[test_index]


#**Splitting Word2Vec Embeddings into Train and Test Datasets

In [ ]:
ai_generated_dataset['word2vecVector'] = ai_generated_dataset['word2vecVector'].apply(lambda x: np.array(x))

X_word2vec = np.stack(ai_generated_dataset['word2vecVector'].values)

X_train_word2vec, X_test_word2vec = X_word2vec[train_index], X_word2vec[test_index]

print(X_train_word2vec.shape)
print(X_test_word2vec.shape)
print(X_train_word2vec)

(35894, 100)
(8974, 100)
[[-0.65024245  0.19181332  1.074703   ... -0.21261919 -0.3324465
  -0.93674195]
 [-0.5001793  -0.08839099  0.37491983 ... -0.7419742  -0.484278
  -0.16304582]
 [-0.7284487   0.3253978   0.8758696  ... -0.20074868 -0.07075012
  -0.23648816]
 ...
 [-0.65365165  0.11777831  0.85443723 ...  0.01597217 -0.9088834
  -0.2610579 ]
 [-0.3111492   0.07460503  0.41989803 ...  0.09475005 -0.6905856
  -0.30385342]
 [-0.89676195  0.42966223  0.54343796 ... -0.11098306 -0.37306657
  -0.5420334 ]]


In [ ]:
# train_essays['word2vecVector'] = train_essays['word2vecVector'].apply(lambda x: np.array(x))

# X_word2vec = np.stack(train_essays['word2vecVector'].values)

# X_train_word2vec, X_test_word2vec = X_word2vec[train_index], X_word2vec[test_index]

# print(X_train_word2vec.shape)
# print(X_test_word2vec.shape)
# print(X_train_word2vec)

#**Splitting Doc2Vec Embeddings into Train and Test Datasets**

In [ ]:
ai_generated_dataset['doc2vecVector'] = ai_generated_dataset['doc2vecVector'].apply(lambda x: np.array(x))
X_doc2vec = np.stack(ai_generated_dataset['doc2vecVector'].values)
X_train_doc2vec, X_test_doc2vec = X_doc2vec[train_index], X_doc2vec[test_index]
print(X_train_doc2vec.shape)
print(X_test_doc2vec.shape)
print(X_train_doc2vec)

(35894, 100)
(8974, 100)
[[ 0.21062015 -0.004447   -0.09290063 ...  0.05933606  0.30229187
  -0.0919051 ]
 [ 0.32557538 -0.1682465  -0.32784384 ... -0.11312847  0.43012157
  -0.0958719 ]
 [-0.25527412  1.470126   -0.09860298 ...  0.052782    0.4983421
   0.10903812]
 ...
 [ 0.08519941  0.04172662 -0.36781234 ... -0.09208106  0.08778302
  -0.14463738]
 [ 0.15849286  0.24429685  0.02958919 ... -0.2148677   0.05409148
  -0.24303465]
 [ 0.13247812 -0.03046415 -0.23711872 ... -0.08562817  0.2642509
  -0.27570826]]


In [ ]:
# train_essays['doc2vecVector'] = train_essays['doc2vecVector'].apply(lambda x: np.array(x))
# X_doc2vec = np.stack(train_essays['doc2vecVector'].values)
# X_train_doc2vec, X_test_doc2vec = X_doc2vec[train_index], X_doc2vec[test_index]
# print(X_train_doc2vec.shape)
# print(X_test_doc2vec.shape)
# print(X_train_doc2vec)

#**Implementing SVM on Different Types of Embeddings and Comparing Results**

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def evaluate_svm_model(X_train, X_test, y_train, y_test):
    svm_model = SVC(class_weight='balanced')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }


results_tfidf = evaluate_svm_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for SVM:", results_tfidf)


results_word2vec = evaluate_svm_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for SVM:", results_word2vec)


results_doc2vec = evaluate_svm_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for SVM:", results_doc2vec)


TF-IDF Results for SVM: {'Accuracy': 0.9936483173612659, 'Precision': 0.9973943254198031, 'Recall': 0.9862582307472088, 'F1-Score': 0.9917950194328488}
Word2Vec Results for SVM: {'Accuracy': 0.9856251392912859, 'Precision': 0.9855658198614319, 'Recall': 0.977383338104781, 'F1-Score': 0.9814575247951703}
Doc2Vec Results for SVM: {'Accuracy': 0.9338087809226655, 'Precision': 0.9171223021582734, 'Recall': 0.9123962210134555, 'F1-Score': 0.9147531572904707}


#**Implementing Decision Tree on Different Types of Embeddings and Comparing Results**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
def evaluate_decisionTree_model(X_train, X_test, y_train, y_test):
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    y_pred = dt_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }

results_tfidf = evaluate_decisionTree_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for Decision Tree:", results_tfidf)


results_word2vec = evaluate_decisionTree_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for Decision Tree:", results_word2vec)


results_doc2vec = evaluate_decisionTree_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for Decision Tree:", results_doc2vec)


TF-IDF Results for Decision Tree: {'Accuracy': 0.9248941386226878, 'Precision': 0.906313058518305, 'Recall': 0.9000858860578299, 'F1-Score': 0.9031887388681413}
Word2Vec Results for Decision Tree: {'Accuracy': 0.9338087809226655, 'Precision': 0.9102745542032267, 'Recall': 0.9206985399370169, 'F1-Score': 0.9154568744662681}
Doc2Vec Results for Decision Tree: {'Accuracy': 0.6445286382883887, 'Precision': 0.5434470891884141, 'Recall': 0.542513598625823, 'F1-Score': 0.5429799426934098}


#**Implementing Random Forest on Different Types of Embeddings and Comparing Results**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def evaluate_randomForest_model(X_train, X_test, y_train, y_test):
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }
results_tfidf = evaluate_randomForest_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for Random Forest:", results_tfidf)


results_word2vec = evaluate_randomForest_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for Random Forest:", results_word2vec)


results_doc2vec = evaluate_randomForest_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for Random Forest:", results_doc2vec)

TF-IDF Results for Random Forest: {'Accuracy': 0.9813906841987965, 'Precision': 0.9914302600472813, 'Recall': 0.960492413398225, 'F1-Score': 0.9757161553002763}
Word2Vec Results for Random Forest: {'Accuracy': 0.9749275685313127, 'Precision': 0.9820058997050147, 'Recall': 0.953048955052963, 'F1-Score': 0.9673107656545111}
Doc2Vec Results for Random Forest: {'Accuracy': 0.8111210162692222, 'Precision': 0.9024171888988362, 'Recall': 0.5771543086172345, 'F1-Score': 0.7040335254059717}


#**Implementing KNN on Different Types of Embeddings and Comparing Results**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# import matplotlib.pyplot as plt
def evaluate_knn_model(X_train, X_test, y_train, y_test,n):
    knn_model = KNeighborsClassifier(n_neighbors=n)
    knn_model.fit(X_train, y_train)
    y_pred = knn_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }


# accuracies = []

# for i in range(1,11):
results_tfidf = evaluate_knn_model(X_train_tfidf, X_test_tfidf, y_train, y_test,3)
print("TF-IDF Results for KNN:", results_tfidf)
# accuracies.append(1-results_tfidf["Accuracy"])


# plt.plot(range(1,11),accuracies)
# plt.xlabel("K")
# plt.ylabel("Error Rate")



results_word2vec = evaluate_knn_model(X_train_word2vec, X_test_word2vec, y_train, y_test,3)
print("Word2Vec Results for KNN:", results_word2vec)


results_doc2vec = evaluate_knn_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test,3)
print("Doc2Vec Results for KNN:", results_doc2vec)



TF-IDF Results for KNN: {'Accuracy': 0.9808335190550479, 'Precision': 0.9767441860465116, 'Recall': 0.9739478957915831, 'F1-Score': 0.9753440366972477}
Word2Vec Results for KNN: {'Accuracy': 0.9854022732337865, 'Precision': 0.992096018735363, 'Recall': 0.9702261666189522, 'F1-Score': 0.9810392242003184}
Doc2Vec Results for KNN: {'Accuracy': 0.8064408290617339, 'Precision': 0.7366576819407008, 'Recall': 0.7824219868308044, 'F1-Score': 0.758850478967097}


#**Implementing the best embedding technique for each model and using it on custom data**


#1.   SVM :- TF-IDF
#2.   Decision Tree :- Word2Vec
#3.   Random Forest :- TF-IDF
#4.   KNN :- Word2Vec  



In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train_tfidf, y_train)
y_pred = knn_model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))


Accuracy: 0.9808335190550479
Precision: 0.9767441860465116
Recall: 0.9739478957915831
F1-Score: 0.9753440366972477
Prediction: [1]


In [ ]:
custom_data=["The beauty of nature lies in its simplicity. Mountains, rivers, forests—they all hold a delicate balance.",
    "Preservation of biodiversity ensures that ecosystems remain sustainable, enabling natural processes such as pollination.",
    "With the advent of smartphones, communication has become more accessible than ever. However, this technological leap comes with its own set of challenges.",
    "The integration of artificial intelligence into daily life has led to innovations like smart homes and autonomous vehicles.",
    "Education is the cornerstone of progress. It's not just about degrees but the ability to think critically.",
    "Online learning platforms have democratized education, allowing individuals from remote areas to access quality resources."]
preprocessed_custom_data = [preprocess(text) for text in custom_data]
custom_data_tfidf = tfidf_vectorizer.transform([custom_data])
prediction = knn_model.predict(custom_data_tfidf)
print("Prediction:", prediction)

Prediction: [0]
